In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io as sio
import librosa
import pdb
import string
import os
import random

!pip install python-levenshtein

from Levenshtein import distance

In [ ]:
def wav2feat(wavfile):
    '''
    Input: audio wav file name
    Output: Magnitude spectrogram
    '''
    x, Fs = librosa.load(wavfile, sr=44100, mono=True) 
    hop = int(0.01 * Fs) # 10ms
    win = int(0.02 * Fs) # 20ms
    X = librosa.stft(x, n_fft=1024, hop_length=hop, win_length=win, window='hann', center=True, pad_mode='reflect')
    return np.abs(X)

def wavs2feat(wavfiles):
    '''
    Concatenate the audio files listed in wavfiles
    Input: list of audio wav file names
    Output: Magnitude spectrogram of concatenated wav
    '''
    x = []
    for wf in wavfiles:
        x1, Fs = librosa.load(wf, sr=44100, mono=True)
        x.append(x1)
    x = np.hstack(x)
    hop = int(0.01 * Fs) # 10ms
    win = int(0.02 * Fs) # 20ms
    X = librosa.stft(x, n_fft=1024, hop_length=hop, win_length=win, window='hann', center=True, pad_mode='reflect')
    return np.abs(X)

def read_csv(filename):
    id_label = {}
    with open(filename,'r') as fid:
        for line in fid: # '176787-5-0-27.wav,engine_idling\n'
            tokens = line.strip().split(',') # ['176787-5-0-27.wav', 'engine_idling']
            id_label[tokens[0]] = tokens[1]
    return id_label

def editDistance(gt, est):
    '''both are lists of labels
    E.g. gt is "dog_bark-street_music-engine_idling"
    E.g. est is "street_music-engine_idling"
    '''
    gttokens = gt.split('-')
    esttokens = est.split('-')
    # Map token to char
    tokenset = list(set(gttokens+esttokens)) # ['dog_bark', 'siren', 'street_music', 'engine_idling']
    token_char = {}
    for i in range(len(tokenset)):
        token_char[tokenset[i]] = string.ascii_uppercase[i]  # {'dog_bark': 'A', 'siren': 'B', 'street_music': 'C', 'engine_idling': 'D'}
    # convert gt and est to strings
    gtstr = [token_char[t] for t in gttokens]
    gtstr = ''.join(gtstr)  # 'BCA'
    eststr = [token_char[t] for t in esttokens]
    eststr = ''.join(eststr)  # 
    # Compare
    editdist = distance(gtstr, eststr) # 1
    score = max(0, 1 - editdist/len(gtstr))
    return editdist, score

def evals(gtcsv, estcsv, taskid):
    gt_id_label = read_csv(gtcsv)
    est_id_label = read_csv(estcsv)
    score = 0
    for id in est_id_label:
        if taskid==1:
            if est_id_label[id] == gt_id_label[id]:
                score += 1
        elif taskid==2:
            _, ss = editDistance(gt_id_label[id], est_id_label[id])
            score += ss
        else:
            pdb.set_trace()
            assert False, ["taskid not correct; it is", taskid]
    avgScore = score/len(est_id_label)
    return avgScore

# if __name__=="__main__":
    # wavs = ['../shared_train/audio_train/180937-7-3-27.wav']
    # wavs = ['/content/drive/MyDrive/Training data/audio_train/100652-3-0-0.wav']
    # print(wavs2feat(wavs).shape)
    # wavfiles = ['../shared_train/audio_train/180937-7-3-27.wav','../shared_train/audio_train/180937-7-3-27.wav']
    # X = wavs2feat(wavs)
    # eval('test_task1/labels.csv', 'test_task1/est.csv', 1)
    # editDistance("dog_bark-street_music-engine_idling","siren-street_music-engine_idling")

#**TASK 2**

##Data Extraction

In [ ]:
# root = '/content/drive/MyDrive/Project/MLSP Course Project/audio_train_1ch/'
# labels = pd.read_csv('/content/drive/MyDrive/Project/MLSP Course Project/labels_train.csv')
root = '/content/drive/MyDrive/Training data/audio_train_1ch/'
df = pd.read_csv('/content/drive/MyDrive/Training data/labels_train.csv')
filename = list(df['slice_file_name'])
classes = list(df['class'])
dic = {}    #contains the file name along with their labels
for i in filename:
  for val in classes:
    dic[i] = val
    classes.remove(val)
    break
enc = {'air_conditioner': 0, 'car_horn': 1, 'children_playing': 2, 'dog_bark': 3, 'drilling': 4, 'engine_idling': 5, 'gun_shot': 6, 'jackhammer': 7, 'siren': 8, 'street_music': 9}
finwav = []
finlab = []
N = 500 #Number of samples
for i in range(N):
  n = np.random.randint(2,6)
  files = []
  prev = []
  wav = []
  labels = []
  for i in range(n):
    wav,label = random.choice(list(dic.items()))
    if len(prev) == 0:
      prev.append(label)
    elif prev[0] == label:
      while(prev[0] == label):
        wav,label = random.choice(list(dic.items()))
      prev = []
      prev.append(label)  
    else:
      prev = []
      prev.append(label)  
    files.append(wav)
    labels.append(enc[label])  
  wavfile = []
  for i in range(n):
    wavfile.append(root+files[i])
  finwav.append(wavfile)
  finlab.append(labels)  

In [ ]:
#Creating the feature array
X_data = []
for i in range(len(finwav)):
  sample = wavs2feat(finwav[i])
  concan = np.zeros((513,2005-sample.shape[1]))
  sample = np.concatenate((sample, concan), axis=1)
  X_data.append(sample)
  print(i)
X_data = np.array(X_data)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
#Creating the target array
Y_data = []
for i in range(len(finlab)):
  temp = np.zeros((10,5))
  # c = 0
  for j in range(len(finlab[i])):
    temp[finlab[i][j]][j] = 1
  Y_data.append(temp.T)
Y_data = np.array(Y_data)

In [ ]:
X_data = X_data.reshape(X_data.shape[0],X_data.shape[1],X_data.shape[2],1)

##Training

In [ ]:
from keras.layers import Bidirectional, TimeDistributed, Conv2D, MaxPooling2D, Input, GRU, Dense, Activation, Dropout, Reshape, Permute
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from sklearn.metrics import confusion_matrix

In [ ]:
def get_model(data_in, data_out, _cnn_nb_filt, _cnn_pool_size, _rnn_nb, _fc_nb):

    #spec_start = Input(shape=(data_in.shape[-3], data_in.shape[-2], data_in.shape[-1]))
    
    spec_start = Input(shape=(data_in.shape[-3], data_in.shape[-2], data_in.shape[-1]))
    spec_x = spec_start

    for _i, _cnt in enumerate(_cnn_pool_size):
        spec_x = Conv2D(filters=_cnn_nb_filt, kernel_size=(3, 3), padding='same')(spec_x)
        spec_x = BatchNormalization(axis=1)(spec_x)
        spec_x = Activation('relu')(spec_x)
        spec_x = MaxPooling2D(pool_size=(1, _cnn_pool_size[_i]* _cnn_pool_size[_i]))(spec_x)
        spec_x = Dropout(0.3)(spec_x)
    spec_x = Permute((2, 1, 3))(spec_x)
    spec_x = Reshape((spec_x.shape[-3], spec_x.shape[-2]*spec_x.shape[-1]))(spec_x)

    for _r in _rnn_nb:
        spec_x = Bidirectional(
            GRU(_r, activation='tanh', dropout=0.3, recurrent_dropout=0.3, return_sequences=True),merge_mode='mul')(spec_x)

    for _f in _fc_nb:
        spec_x = TimeDistributed(Dense(_f))(spec_x)
        spec_x = Dropout(0.3)(spec_x)

    spec_x = TimeDistributed(Dense(data_out.shape[-1]))(spec_x)
    out = Activation('softmax', name='strong_out')(spec_x)

    model = Model(inputs=spec_start, outputs=out)
    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
model = get_model(X_data,Y_data,16,[5,2,2],[8,8],[8])
history = model.fit(x=X_data, y=Y_data, epochs = 100, shuffle = True)

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 513, 2005, 1)]    0         
_________________________________________________________________
conv2d (Conv2D)              (None, 513, 2005, 16)     160       
_________________________________________________________________
batch_normalization (BatchNo (None, 513, 2005, 16)     2052      
_________________________________________________________________
activation (Activation)      (None, 513, 2005, 16)     0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 513, 80, 16)       0         
_________________________________________________________________
dropout (Dropout)            (None, 513, 80, 16)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 513, 80, 16)      

In [ ]:
# model.save('/content/drive/MyDrive/Training data/My model1')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Training data/My model1/assets


##Test data

In [ ]:
# root1 = '/content/drive/MyDrive/Project/MLSP Course Project/sample_test_task1/feats/'
# df1 = pd.read_csv('/content/drive/MyDrive/Project/MLSP Course Project/sample_test_task1/labels.csv')
# root1 = '/content/drive/MyDrive/Training data/sample_test_task2/feats/'
# df1 = pd.read_csv('/content/drive/MyDrive/Training data/sample_test_task2/labels.csv')
# df1 = np.array(df1)
# test_data = []
# sample = np.load(root1 + 'a00001.npy') 
# concan = np.zeros((513,2005-sample.shape[1]))
# sample = np.concatenate((sample, concan), axis=1)
# test_data.append(sample)
# i = 1
# for file in df1[:,:1]:
#   sample = np.load(root1 + file[0] + '.npy') 
#   concan = np.zeros((513,2005-sample.shape[1]))
#   sample = np.concatenate((sample, concan), axis=1)
#   test_data.append(sample)
#   print(i)
#   i = i+1

test_data = []
for i in range(1,31):
  if (i<10):
    path='/content/drive/MyDrive/Training data/test_task2/feats/a00'+str(i)+'.npy'
  else:
    path='/content/drive/MyDrive/Training data/test_task2/feats/a0'+str(i)+'.npy'
  spect=np.load(path)
  concan = np.zeros((513,2005-spect.shape[1]))
  sample = np.concatenate((spect, concan), axis=1)
  test_data.append(sample)
test_data = np.array(test_data)
X_test = test_data.reshape(test_data.shape[0], 513, 2005, 1)

In [ ]:
pred = model.predict(X_test)
pred = np.argmax(pred,axis=2)

cl = list(enc.keys())
la = list(enc.values())
 
Y_pred = []         #list of predicted labels
for i in range(len(pred)):
  lab = ''
  prev = []
  for j in range(5):
    position = la.index(pred[i][j])
    if len(prev) == 0:
      prev.append(cl[position])
    elif prev[0] == cl[position]:
      continue
    else:
      prev = []
      prev.append(cl[position])    
    lab += cl[position] + '-'
  lab = lab[:-1]
  Y_pred.append(lab)

##Result

In [ ]:
fname = []
for i in range(1,31):
  if (i<10):
    temp_str='a00'+str(i)+'.npy'
  else:
    temp_str='a00'+str(i)+'.npy'
  fname.append(temp_str)
dict = {'name': fname, 'label': Y_pred}
df2 = pd.DataFrame(dict)
df2.to_csv('/content/drive/MyDrive/Training data/task3_labels_test.csv', header=False, index=False)

In [ ]:
gtcsv = '/content/drive/MyDrive/Training data/task2_labels_test.csv'
estcsv = '/content/drive/MyDrive/Training data/task3_labels_test.csv'
evals(gtcsv,estcsv,2)

1.0